This notebook aims to find near duplicates using the SimHash technique, and organizing them in an M-Tree

In [6]:
# import similarities.mtree
import importcsv
from similarities import libsim, mtree
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
print('Importing csv file...........')
small_dataset = importcsv.opencsv('processedspeechfinal.csv') 

Importing csv file...........


Κρατάω μόνο τα processed speeches

In [5]:
speeches = small_dataset['processed_speeches']
# speeches = small_dataset['speech']

In [4]:
print(speeches)
print(type(speeches[0]))
# speeches.head()

0         ΠΑΡΑΚΑΛΕΙΤΑ ΚΥΡΙ ΓΡΑΜΜΑΤΕ ΣΥΝΟΔΕΥΣ ΙΕΡ ΣΥΝΟΔ Α...
1         ΥΠΑΡΧ ΕΚ ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΨΗΦΙΣ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩΜ...
2         ΚΥΡΙ ΒΟΥΛΕΥΤ ΤΙΜ ΑΝΑΚΟΙΝΩΣ ΣΩΜ ΒΟΥΛΕΥΤ ΕΒΡ ΥΠΟ...
3         ΥΠΑΡΧ ΕΚ ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΕΨΗΦΙΣ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩ...
4         ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΤΙΜ ΑΝΑΚΟΙΝΩΣ ΑΠΟΤΕΛΕΣΜ ΔΙΕΞΑΧΘΕ...
                                ...                        
362625    Π ΤΕΘ ΠΕΡΙΟΡΙΣΜ ΠΑΜ ευρωπαϊκη ΕΝΩΣ ΖΗΤΑΜ ΛΕΜ ...
362626    ΥΠΗΡΧΕΕΜ ΑΝΑΠΤΥΞΙΑΚ ΥΠΟΥΡΓΕΙ ΥΠΟΥΡΓΕΙ ΚΑΝ ΚΟΙΝ...
362627    ΕΥΧΑΡΙΣΤΟΥΜ ΥΠΟΥΡΓ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩΜΕΝ ΣΥΖΗΤΗΣ...
362628    ΟΛΟΚΛΗΡΩΣ ΨΗΦΟΦΟΡΙ ΗΛΕΚΤΡΟΝΙΚ ΣΥΣΤΗΜ ΣΧΕΔΙ ΝΟΜ...
362629    ΣΥΝΑΙΝΕΣ ΣΩΜΑΤ ΩΡ 1949΄ ΛΥΕΤΑ ΣΥΝΕΔΡΙΑΣ ΔΕΥΤΕΡ...
Name: processed_speeches, Length: 362630, dtype: object
<class 'str'>


We need to define the weights of each word for all speeches, and keep them in a dictionary.
The dictionary is of the form {word: weight}

### Find word weights

In [6]:
# This shall come in handy
# https://www.reddit.com/r/LanguageTechnology/comments/dugjis/approximate_string_matching_with_simhash_and_a/
# def tokenizer(word):
#     return word.split()
# del vectorizer, X
# speeches = speeches.to_list()
vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=False, )
X = vectorizer.fit_transform(speeches.values)

In [7]:
names = vectorizer.get_feature_names_out()
index = 10550

print(type(names))


<class 'numpy.ndarray'>


We need word weights for this to work

In [8]:
max_idf = len(names) -1
weights = {name: (idx/max_idf) for idx, name in enumerate(names)} # normalized weights
weights

{'0000τοσο': 0.0,
 '0001ευχαριστω': 1.5693635132399353e-06,
 '0002γραφημα': 3.1387270264798705e-06,
 '0002θεωρουμε': 4.708090539719806e-06,
 '0003επιλεξει': 6.277454052959741e-06,
 '0004γων': 7.846817566199676e-06,
 '0006που': 9.416181079439611e-06,
 '0009ορισμενεσ': 1.0985544592679547e-05,
 '000αυτα': 1.2554908105919482e-05,
 '000δρχ': 1.4124271619159418e-05,
 '000καταστηματαρχεσ': 1.569363513239935e-05,
 '0010νομουσ': 1.726299864563929e-05,
 '0011δυο': 1.8832362158879222e-05,
 '0012αναφερθω': 2.040172567211916e-05,
 '0012βητηση': 2.1971089185359093e-05,
 '0013απο': 2.354045269859903e-05,
 '0013δεν': 2.5109816211838964e-05,
 '0014περιοριζετε': 2.6679179725078898e-05,
 '0016των': 2.8248543238318835e-05,
 '0018ξεφορτωνονται': 2.981790675155877e-05,
 '0019τονουσ': 3.13872702647987e-05,
 '001κυριε': 3.295663377803864e-05,
 '0021γιατι': 3.452599729127858e-05,
 '0021μπορουμε': 3.609536080451851e-05,
 '0021χωσ': 3.7664724317758445e-05,
 '0022μικρο': 3.9234087830998385e-05,
 '0022πρου': 4.080

Acquire the signature for one document

In [9]:
import importlib
importlib.reload(libsim)

sig = libsim.getSignatureOfSpeech(speeches[0], weights)
print('signature of first document:',bin(sig))

signature of first document: 0b1001011010111010010101001100100100000110101101000101010010111101


### Create M-Tree strucure
This data structure will hold all the signatures, based on their Hamming Distance.
This way, we can find the relevant ones

In [ ]:
# The following list is an array of signatures in integer form 
# signatures = [libsim.getSignatureOfSpeech(speech, weights) for speech in speeches]
signatures = {idx: libsim.getSignatureOfSpeech(speech, weights) for idx, speech in enumerate(speeches)}
# this took 14 minutes to complete

In [11]:
# SAVE SPEECHES DICTIONARY 
# run the following code? protection against Run-All
if 1 == 0: # change this if statement
    import pickle
    filename = 'speeches_signatures.pkl'
    with open(filename, 'wb') as fo:
        pickle.dump(signatures, fo)

In [2]:
# LOAD SIGNATURES DICTIONARY
# run the following code? protection against Run-All
if 1 == 0: # change this if statement
    import pickle
    filename = 'speeches_signatures.pkl'
    with open(filename, 'rb') as fo:
        signatures = pickle.load(fo) # this is a dictionary of {indexOfSpeech: signatureOfSpeech}
        

In [4]:
def hamming_distance(a: int, b:int):
    """
    This function returns the hamming distance between two integers.
    The hamming distance is defined as the total number of 1s, after
    the XOR operation between two binary numbers
    """
    return int.bit_count(a ^ b) # requires python 3.10

In [3]:
from collections import defaultdict

inv_signatures = defaultdict(list)
for k, v in signatures.items():
    inv_signatures[v].append(k)
# inv_signatures = {v: k for k,v in signatures.items()}

duplicates = 0
for k,v in inv_signatures.items():
    if len(v) > 1:
        duplicates += 1
        print(k, v)

print(duplicates)

9915691404370057697 [3620, 8820]
11441481318061656063 [4881, 13461, 216325, 232267]
9276273966249442966 [7347, 13259, 13287, 211886]
10337763429474026545 [9415, 41556, 162364, 264154, 274587]
10337763429474030641 [16478, 264814]
11199746627117819910 [22631, 22748]
11738757305857261878 [26714, 26745]
9484019178115602237 [26715, 26746]
10288825766698324407 [26717, 26748]
9271619591175409998 [32496, 32498]
16800802462426390460 [37489, 128532, 171891]
10008146449412785897 [45776, 70715, 70716, 256337]
9368319034869083329 [57138, 57214]
9982255801337708502 [57140, 57216]
9550820832545888508 [57141, 57217]
9278896845068334028 [57143, 57219]
9277820288899890817 [57146, 57222]
10225521772886555614 [57147, 57223]
9551100876427755478 [57149, 57225]
10007818224999055052 [57153, 57229]
9863572074902896342 [57154, 57230]
9776010378616772994 [57155, 57231]
10009887401288095703 [57157, 57233]
9998778423717235328 [57158, 57234]
11153241996440029575 [57160, 57236]
9854235128240360148 [57162, 57238]
991

In [11]:
# SAVE INVERTED SIGNATURES DICTIONARY
# run the following code? protection against Run-All
if 1 == 0: # change this if statement
    import pickle
    filename = 'speeches_inv_signatures.pkl'
    with open(filename, 'wb') as fo:
        pickle.dump(inv_signatures, fo)

In [1]:
# LOAD INVERTED SIGNATURES DICTIONARY
# run the following code? protection against Run-All
if 1 == 0: # change this if statement
    import pickle
    filename = 'speeches_inv_signatures.pkl'
    with open(filename, 'rb') as fo:
        inv_signatures = pickle.load(fo) # this is a dictionary of {indexOfSpeech: signatureOfSpeech}

In [10]:
print(signatures[0])

10861086673735013565


In [ ]:
"""
# find values that have the same key
def find_duplicate_values(dictionary):
    value_count = {}
    duplicates = []

    for key, value in dictionary.items():
        if value in value_count:
            duplicates.append(value)
        else:
            value_count[value] = key

    return duplicates

result = find_duplicate_values(signatures)
if result:
    print(f"The following values have multiple keys pointing to them: {result}")
else:
    print("No duplicate values found.")
"""

In [4]:
print(inv_signatures[10861086673735013565])

[0]


In [11]:
# Found help here
# https://www.geeksforgeeks.org/multithreading-in-python-set-2-synchronization/

import threading
import time
# I'm gonna make you run in parallel you stupid bitch
final_dict = dict()

def thread_task(lock, start, end):
    # we already have the signatures dictionary
    # let's slice it accordingly
    print(f'Start: {start}, end {end}', flush=True)
    subdict = dict(list(signatures.items())[start:end])
    for index, signature in subdict.items():
        if index % ((end-start) // 20) == 0:
            print(index, time.localtime())
        res = list(tree.search(signature, 4)) # for the given signature, find 3 similar speeches
    
        similar_speech_1_index = inv_signatures[res[0]][0] # probably the speech itself!
        similar_speech_2_index = inv_signatures[res[1]][0] if len(inv_signatures[res[1]]) > 0 else -1
        similar_speech_3_index = inv_signatures[res[2]][0] if len(inv_signatures[res[2]]) > 0 else -1
        similar_speech_4_index = inv_signatures[res[3]][0] if len(inv_signatures[res[3]]) > 0 else -1
        
        # safe multithreading!
        lock.acquire()
        final_dict[index] = [similar_speech_1_index, similar_speech_2_index,similar_speech_3_index, similar_speech_4_index]
        lock.release()

def main_task():
    threads = []
    lock = threading.Lock()
    splits = 5
    step = len(signatures) // splits
    i = 0
    for _ in range(splits+1): # add 1 because there are 4 remainder
        start = i
        end = i + step
        thread = threading.Thread(target=thread_task, args=(lock, start, end))
        threads.append(thread)
        i = step + i

    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

if __name__ == '__main__':
    main_task()
juice = """
for index, signature in signatures.items():
    if (index % 10000 == 0):
        print(index, len(signatures))
    res = list(tree.search(signature, 4)) # for the given signature, find 3 similar speeches
    
    similar_speech_1_index = inv_signatures[res[0]][0] # probably the speech itself!
    similar_speech_2_index = inv_signatures[res[1]][0] if len(inv_signatures[res[1]]) > 0 else -1
    similar_speech_3_index = inv_signatures[res[2]][0] if len(inv_signatures[res[2]]) > 0 else -1
    similar_speech_4_index = inv_signatures[res[3]][0] if len(inv_signatures[res[3]]) > 0 else -1
    
    final_dict[index] = [similar_speech_1_index, similar_speech_2_index,]
"""

64566
129132
193698
258264
322830
387396


In [ ]:
with open('final_dictionary.pkl', 'wb') as fo:
    pickle.dump(final_dict, fo)

In [22]:
print(inv_signatures[11210660306253846063])

[152]


#### The magic number 3
If the distance between two document signatures is greater than 3, the documents are not near duplicates. This is taken as-is, no further investigation is needed. (Apostolos Papadopoulos)

In [7]:
tree = mtree.MTree(hamming_distance, max_node_size=10)
tree.add_all(signatures.values())
# takes 20 seconds to build the tree

In [10]:
# SAVE SIMILARITIES TREE
if 1 == 0: # protection against Run-All operations
    import pickle
    with open('sim_tree.pkl', 'wb') as fo:
        pickle.dump(tree,fo)

In [5]:
# LOAD SIMILARITIEES TREE
# run the following code? protection against Run-All
if 1 == 0: # change this if statement
    import pickle
    filename = 'sim_tree.pkl'
    with open(filename, 'rb') as fo:
        tree = pickle.load(fo)

In [15]:
m = tree.search(signatures[1], 3) # this returns a map object

Let's see if a query is possible

In [16]:
list(m)

[12369499147591332851, 12657869161718993890, 12153891273390535635]

### We have a tree of signatures
Now all the signatures are stored in a dictionary, indexed based on speech

# Load all data - useful for debugging
The following cell is a fast way to load all necessary data for this notebook

In [2]:
def hamming_distance(a: int, b:int):
    """
    This function returns the hamming distance between two integers.
    The hamming distance is defined as the total number of 1s, after
    the XOR operation between two binary numbers
    """
    return int.bit_count(a ^ b) # requires python 3.10
# LOAD ALL DATA
# LOAD SIMILARITIEES TREE
# run the following code? protection against Run-All
if 0 == 0: # change this if statement
    import pickle
    filename = 'sim_tree.pkl'
    with open(filename, 'rb') as fo:
        tree = pickle.load(fo)

    # LOAD SIGNATURES DICTIONARY
    # run the following code? protection against Run-All
    filename = 'speeches_inv_signatures.pkl'
    with open(filename, 'rb') as fo:
        inv_signatures = pickle.load(fo) # this is a dictionary of {indexOfSpeech: signatureOfSpeech}

    # LOAD SIGNATURES DICTIONARY
    # run the following code? protection against Run-All
    filename = 'speeches_signatures.pkl'
    with open(filename, 'rb') as fo:
        signatures = pickle.load(fo) # this is a dictionary of {indexOfSpeech: signatureOfSpeech}
